# 06. Solucionador del TSP del metro de CDMX con Búsqueda Tabu

Carga de librerías

In [1]:
# Data
import json

# Manipulation
import pandas as pd

# Graph
import networkx as nx
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import plotly.graph_objects as go

# Utils
from random import shuffle, sample
from time import perf_counter_ns


Cargar el grafo del metro y los tiempos de viaje

In [2]:
metro_graph = pickle.load(open("../../output_metro/metro_graph.pickle", "rb"))

# Tiempos de viaje
json_file = "../../output_metro/travel_times_metro.json"
with open(json_file) as input_json:
    dict_times_metro = json.load(input_json)

Crear el diccionario de ubicaciones de estaciones

In [3]:
location_stations = dict()
for route_id in dict_times_metro.keys():
    stations_data = dict_times_metro[route_id]
    for station_data in stations_data:
        location_stations[station_data[0]] = tuple(station_data[2:])

Se realiza una implementación de la metaheurística __Búsqueda Tabú__ para hallar soluciones sobre el grafo completo en base a su matriz de distancias.

In [4]:
class TSP_TabuSearch:
    def __init__(self, graph: nx.Graph, nodes_to_visit: list[str], tabu_size: int, max_iters: int):

        """
        Initializes the simulated annealing algorithm for solving the Traveling Salesman Problem (TSP).

        Parameters:
        graph (nx.Graph): A Graph containing nodes, edges and weights for each edge
        distance_matrix (pd.DataFrame): A DataFrame containing the distances between nodes.
        nodes_to_visit (list[str]): A list of nodes to be visited.
        init_temp (float): Initial temperature for the annealing process.
        max_iters (int): Maximum number of iterations to perform.
        tabu_list (list): List of tabu values.
        """

        self.__graph = graph
        self.__dist_matrix = self.__get_distances_matrix()
        self.__nodes_to_visit = nodes_to_visit
        self.__tabu_size = tabu_size
        self.__max_iters = max_iters
        self.__size_nodes = len(nodes_to_visit)
        self.tabu_list = []
        
    def __get_distances_matrix(self):

        """
        Computes the distances matrix for all pairs of nodes in the graph.

        This private method generates a matrix where each element represents the shortest path 
        distance between a pair of nodes in the graph. The distances are computed using Dijkstra's 
        algorithm, assuming the graph is weighted.

        Returns:
            pd.DataFrame: A DataFrame where the rows and columns correspond to the graph's nodes, 
                          and each element [i, j] contains the shortest path distance from node i to node j.

        Notes:
            - The method utilizes NetworkX's shortest_path_length function with a weight parameter 
              to account for edge weights.
            - The resulting DataFrame is symmetric if the graph is undirected, with zeros on the diagonal.
        """

        matrix_distance = pd.DataFrame(index=self.__graph.nodes(), columns=self.__graph.nodes())
        for origin_node in self.__graph.nodes():
            length = nx.single_source_dijkstra_path_length(self.__graph, origin_node, weight='weight')
            for target_node, distance in length.items():
                matrix_distance.at[origin_node, target_node] = distance
        return matrix_distance.astype(float)
    
    def __generate_initial_path(self):
        """
        Generates the initial path by shuffling the list of nodes to visit.

        Returns:
        list: A shuffled list representing the initial path.
        """
        path = self.__nodes_to_visit[:]
        shuffle(path)
        return path

    def __generate_neighbors(self, path):
        """
        Generates neighbors by swapping a pair of nodes.

        Returns:
        list: A  list of neighbours.
        """
        neighbors = []
        for i in range(self.__size_nodes):
            for j in range(i+1, self.__size_nodes):
                neighbor = path[:]
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        return neighbors

    def __compute_path_cost(self, path):
        """
        Computes the total cost of the given path based on the distance matrix.

        Parameters:
        path (list): The path for which the cost is to be computed.

        Returns:
        float: The total cost of the path.
        """
        total_cost = 0.0
        for k in range(len(path) - 1):
            total_cost += self.__dist_matrix.loc[path[k], path[k+1]]
        total_cost += self.__dist_matrix.loc[path[-1], path[0]]
        return total_cost

    def __get_full_tsp_path(self):
        """
        Converts the best path found into a full path including all intermediate nodes.

        This method uses the shortest path between each pair of nodes in the best path to
        generate a complete route, ensuring all nodes are visited in sequence.
        """
        complete_best_route = []
        for i in range(self.__size_nodes - 1):
            complete_best_route += nx.shortest_path(self.__graph, self.best_path[i], self.best_path[i+1], weight="weight")
        complete_best_route += nx.shortest_path(self.__graph, self.best_path[-1], self.best_path[0], weight="weight")
        self.best_path = complete_best_route
        self.best_cost = self.__compute_path_cost(self.best_path)

    def find_solution(self):
        """
        Executes the simulated annealing algorithm to find the best solution for the TSP.

        Returns:
        tuple: A tuple containing the best path and the best cost.
        """
        start_time = perf_counter_ns()
        
        self.__current_path = self.__generate_initial_path()
        actual_cost = self.__compute_path_cost(self.__current_path)

        self.best_path = self.__current_path[:]
        self.best_cost = actual_cost

        for iteration in range(self.__max_iters):
            neighbors = self.__generate_neighbors(self.__current_path)
            best_neighbor = None
            best_neighbor_cost = float('inf')

            for neighbor in neighbors:
                if neighbor not in self.tabu_list:
                    neighbor_cost = self.__compute_path_cost(neighbor)
                    if neighbor_cost < best_neighbor_cost:
                        best_neighbor = neighbor
                        best_neighbor_cost = neighbor_cost

            if best_neighbor is None:
                break

            self.__current_path = best_neighbor
            actual_cost = best_neighbor_cost

            if actual_cost < self.best_cost:
                self.best_path = self.__current_path[:]
                self.best_cost = actual_cost

            self.tabu_list.append(self.__current_path)
            if len(self.tabu_list) > self.__tabu_size:
                self.tabu_list.pop(0)

        self.__get_full_tsp_path()
        
        end_time = perf_counter_ns()
        self.perf_time = (end_time - start_time)/1e9

        print(f"Finished Tabu Search with...\n{iteration+1} iterations\nBest Solution: {self.best_path}\nBest Cost: {self.best_cost}")

        return self.best_path, self.best_cost

Planteamos el problema de visitar las estaciones que tengan grado igual a 1; es decir, que solo tengan una estación vecina y esta misma sea la única por la cúal se puede acceder al respectivo nodo. Por propositos de mejor visualización, se retira la estación con etiqueta BUENAVISTA puesto que está se encuentra en el centro del sistema de metro y desfavorecería la tarea de hallar las diferencias entre las soluciones generadas

In [5]:
node_degrees = dict(metro_graph.degree())
nodes_to_visit = [node for node in metro_graph.nodes() if node_degrees[node] == 1 and node != "BUENAVISTA"]


Ejecutar la Búsqueda Tabú

In [36]:
found_paths = []
found_costs = []
perf_times = []
for k in range(5):
    tabu_tsp = TSP_TabuSearch(metro_graph, nodes_to_visit, tabu_size=5, max_iters=5)
    best_solution, best_cost = tabu_tsp.find_solution()
    found_paths.append(best_solution)
    found_costs.append(best_cost)
    perf_times.append(tabu_tsp.perf_time)

# Generar y visualizar una tabla de las 5 rutas generadas acompañadas de su costo
FoundRoutesCost_df = pd.DataFrame({"Costo": found_costs, "Tiempo": perf_times})

MinCostIndex = FoundRoutesCost_df.idxmin()["Costo"]
MinCost = FoundRoutesCost_df.loc[MinCostIndex]["Costo"]
MaxCostIndex = FoundRoutesCost_df.idxmax()["Costo"]
MaxCost = FoundRoutesCost_df.loc[MaxCostIndex]["Costo"]

AverageCost = FoundRoutesCost_df["Costo"].mean()
AverageTime = FoundRoutesCost_df["Tiempo"].mean()

print(f"The best route given by the solver is the #{MinCostIndex} with cost {MinCost}")
print(f"The worst route given by the solver is the #{MaxCostIndex} with cost {MaxCost}")
print(f"The average cost is {AverageCost}")
print(f"The average computing time is {AverageTime}")

FoundRoutesCost_df

Finished Tabu Search with...
5 iterations
Best Solution: ['TLAHUAC', 'TLALTENCO', 'ZAPOTITLAN', 'NOPALERA', 'OLIVOS', 'TEZONCO', 'PERIFERICOOTE', 'CALLE11', 'LOMASESTRELLA', 'SANANDRESTO', 'CULHUACAN', 'ATLALILCO', 'IZTAPALAPA', 'CERROESTRELLA', 'UAM', 'CONST1917', 'CONST1917', 'UAM', 'CERROESTRELLA', 'IZTAPALAPA', 'ATLALILCO', 'MEXICALTZINGO', 'ERMITA', 'EJECENTRAL', 'PARQUEVENADOS', 'ZAPATA', 'HOSPITAL20', 'INSURGENTESSUR', 'MIXCOAC', 'BARRANCA', 'BARRANCA', 'MIXCOAC', 'SANANTONIO', 'SANPEDROPINOS', 'TACUBAYA', 'OBSERVATORIO', 'OBSERVATORIO', 'TACUBAYA', 'CONSTITUYENTES', 'AUDITORIO', 'POLANCO', 'SANJOAQUIN', 'TACUBA', 'PANTEONES', 'CUATROCAMIN', 'CUATROCAMIN', 'PANTEONES', 'TACUBA', 'CUITLAHUAC', 'POPOTLA', 'COLMILITAR', 'NORMAL', 'SANCOSME', 'REVOLUCION', 'HIDALGO', 'GUERRERO', 'TLATELOLCO', 'LARAZA', 'AUTOBUSESNTE', 'ITOPETROLEO', 'POLITECNICO', 'POLITECNICO', 'ITOPETROLEO', 'LINDAVISTA', 'DTVO18MARZO', 'INDIOSVERD', 'INDIOSVERD', 'DTVO18MARZO', 'POTRERO', 'LARAZA', 'TLATELOLCO', 

Costo    Tiempo
0  6.389444  0.017181
1  5.953056  0.018691
2  5.836111  0.016633
3  6.113333  0.019964
4  6.276389  0.019916

In [31]:
found_paths = []
found_costs = []
perf_times = []
for k in range(5):
    tabu_tsp = TSP_TabuSearch(metro_graph, nodes_to_visit, tabu_size=5, max_iters=15)
    best_solution, best_cost = tabu_tsp.find_solution()
    found_paths.append(best_solution)
    found_costs.append(best_cost)
    perf_times.append(tabu_tsp.perf_time)

# Generar y visualizar una tabla de las 5 rutas generadas acompañadas de su costo
FoundRoutesCost_df = pd.DataFrame({"Costo": found_costs, "Tiempo": perf_times})

MinCostIndex = FoundRoutesCost_df.idxmin()["Costo"]
MinCost = FoundRoutesCost_df.loc[MinCostIndex]["Costo"]
MaxCostIndex = FoundRoutesCost_df.idxmax()["Costo"]
MaxCost = FoundRoutesCost_df.loc[MaxCostIndex]["Costo"]

AverageCost = FoundRoutesCost_df["Costo"].mean()
AverageTime = FoundRoutesCost_df["Tiempo"].mean()

print(f"The best route given by the solver is the #{MinCostIndex} with cost {MinCost}")
print(f"The worst route given by the solver is the #{MaxCostIndex} with cost {MaxCost}")
print(f"The average cost is {AverageCost}")
print(f"The average computing time is {AverageTime}")

FoundRoutesCost_df

Finished Tabu Search with...
15 iterations
Best Solution: ['UNIVERSIDAD', 'COPILCO', 'MAQ', 'VIVEROS', 'COYOACAN', 'ZAPATA', 'PARQUEVENADOS', 'EJECENTRAL', 'ERMITA', 'GRALANAYA', 'TASQUENA', 'TASQUENA', 'GRALANAYA', 'ERMITA', 'MEXICALTZINGO', 'ATLALILCO', 'IZTAPALAPA', 'CERROESTRELLA', 'UAM', 'CONST1917', 'CONST1917', 'UAM', 'CERROESTRELLA', 'IZTAPALAPA', 'ATLALILCO', 'CULHUACAN', 'SANANDRESTO', 'LOMASESTRELLA', 'CALLE11', 'PERIFERICOOTE', 'TEZONCO', 'OLIVOS', 'NOPALERA', 'ZAPOTITLAN', 'TLALTENCO', 'TLAHUAC', 'TLAHUAC', 'TLALTENCO', 'ZAPOTITLAN', 'NOPALERA', 'OLIVOS', 'TEZONCO', 'PERIFERICOOTE', 'CALLE11', 'LOMASESTRELLA', 'SANANDRESTO', 'CULHUACAN', 'ATLALILCO', 'ESCUADRON', 'ACULCO', 'APATLACO', 'IZTACALCO', 'COYUYA', 'SNTAANITA', 'JAMAICA', 'MIXIUHCA', 'VELODROMO', 'CIUDADDVA', 'PUEBLA', 'PANTITLAN', 'AGRICOLA', 'CANALSANJUAN', 'TEPALCATES', 'GUELATAO', 'PENONVIEJO', 'ACATITLA', 'STAMARTA', 'LOSREYES', 'LAPAZ', 'LAPAZ', 'LOSREYES', 'STAMARTA', 'ACATITLA', 'PENONVIEJO', 'GUELATAO', '

Costo    Tiempo
0  5.693056  0.045797
1  5.693056  0.046560
2  5.693056  0.046212
3  5.693056  0.048388
4  5.693056  0.051388

In [20]:
tabu_sizes = [5, 10]
max_iterations = [5, 15]

for size in tabu_sizes:
    for iters in max_iterations:
        tabu_tsp = TSP_TabuSearch(metro_graph, nodes_to_visit, tabu_size=size, max_iters=iters)
        best_path, best_cost = tabu_tsp.find_solution()
        print(f"Tabu Size: {size}, Max Iterations: {iters}, Best Cost: {best_cost}")

Finished Tabu Search with...
5 iterations
Best Solution: ['TASQUENA', 'GRALANAYA', 'ERMITA', 'EJECENTRAL', 'PARQUEVENADOS', 'ZAPATA', 'COYOACAN', 'VIVEROS', 'MAQ', 'COPILCO', 'UNIVERSIDAD', 'UNIVERSIDAD', 'COPILCO', 'MAQ', 'VIVEROS', 'COYOACAN', 'ZAPATA', 'HOSPITAL20', 'INSURGENTESSUR', 'MIXCOAC', 'BARRANCA', 'BARRANCA', 'MIXCOAC', 'SANANTONIO', 'SANPEDROPINOS', 'TACUBAYA', 'CONSTITUYENTES', 'AUDITORIO', 'POLANCO', 'SANJOAQUIN', 'TACUBA', 'PANTEONES', 'CUATROCAMIN', 'CUATROCAMIN', 'PANTEONES', 'TACUBA', 'SANJOAQUIN', 'POLANCO', 'AUDITORIO', 'CONSTITUYENTES', 'TACUBAYA', 'OBSERVATORIO', 'OBSERVATORIO', 'TACUBAYA', 'PATRIOTISMO', 'CHILPANCINGO', 'CENTROMEDICO', 'LAZAROCAR', 'CHABACANO', 'JAMAICA', 'MIXIUHCA', 'VELODROMO', 'CIUDADDVA', 'PUEBLA', 'PANTITLAN', 'AGRICOLA', 'CANALSANJUAN', 'TEPALCATES', 'GUELATAO', 'PENONVIEJO', 'ACATITLA', 'STAMARTA', 'LOSREYES', 'LAPAZ', 'LAPAZ', 'LOSREYES', 'STAMARTA', 'ACATITLA', 'PENONVIEJO', 'GUELATAO', 'TEPALCATES', 'CANALSANJUAN', 'AGRICOLA', 'PANTITL

Visualizamos el grafo original del sistema de metro de la Ciudad de México con la ruta solución

In [37]:
class ColorizerByMap:
  def __init__(self, cmap_name, start_val, stop_val):
    self.cmap_name = cmap_name
    self.cmap = plt.get_cmap(cmap_name)
    self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
    self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

  def get_rgb(self, val):
    return self.scalarMap.to_rgba(val, bytes=False, norm=True)

In [38]:
colors_found_solutions = []    
for path in found_paths:   
    size_tsp_path_SA = len(path)
    colorize_tsp_path_SA = ColorizerByMap("coolwarm", 0, size_tsp_path_SA)
    colors_tsp_path = {node:'rgba' + str(colorize_tsp_path_SA.get_rgb(i)) for i, node in enumerate(path)}

    colors_found_solutions.append(colors_tsp_path)

In [39]:
edge_x = []
edge_y = []
for edge in metro_graph.edges():
    x0, y0 = location_stations[edge[0]]
    x1, y1 = location_stations[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

node_x = []
node_y = []
node_text = []
node_degrees = dict(metro_graph.degree())
for node in metro_graph.nodes():
    x, y = location_stations[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"{node}\n# de estaciones: {node_degrees[node]}")

In [40]:
NodesNames = list(metro_graph.nodes())
ResultsOrder = [(k, cost_k) for k, cost_k in enumerate(found_costs)]
ResultsOrder.sort(key = lambda x: x[1])
IndexOrder, _ = zip(*ResultsOrder)

for k in IndexOrder:
    colors_nodes = len(NodesNames)*['rgba(0,0,0,1)']
    for node in found_paths[k]:
        colors_nodes[NodesNames.index(node)] = colors_found_solutions[k][node]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='gray'),
        hoverinfo='none',
        mode='lines')

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_text,
        textposition="top center",
        marker = dict(
            color = colors_nodes,
            size = 10,
            line = dict(
                color = "black",
                width = 1
            )        
        ),
        line_width=2)

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=f'Ruta #{k+1} obtenida por Búsqueda Tabú',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            text="",
                            showarrow=False,
                            xref="paper", yref="paper")],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)))

    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
    )

    fig.show()